In [ ]:
# This work was design to build a rank list based on customer's likelihood on purchasing certain product 
# as an advising reference to marketing groups.
# The file contain all model constructing and evaluation script based on pre-constructed, selected databases
# queried using SQL.

# Notes: Some reference name and specific section of code are removed as the file is only for demonstration purpose

In [ ]:
# Importing library for model visualization, model construction abd evaluation

from sklearn.model_selection import train_test_split,StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
from lightgbm import LGBMClassifier,plot_tree
from sklearn.ensemble import VotingClassifier
import scikitplot as skplt
import plot_metric  #pip install plot-metric
from plot_metric import functions
import graphviz
import sqlalchemy #uses pyodbc or ibm_db backend
import ibm_db
import ibm_db
import ibm_db_sa # uses ibm-db package
import numpy as np
import pandas as pd
import lightgbm
import shap
import re
import pickle
import seaborn as sns
import matplotlib.pyplot as  plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 1100)
#pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Import data file
df_samp =pd.read_pickle('DataFile_Path')#retrieve data from the pickle file
df_samp.columns= df_samp.columns.str.upper()

In [ ]:
#score data
df_score =pd.read_pickle("DataFile_Path") #retrieve data from the pickle file
df_score.columns= df_score.columns.str.upper()
df_score.shape

In [ ]:
dataset=df_samp[columns.upper().split()]

dataset.dtypes

* Change the data type of below features to float

In [ ]:
columns_change_dtype=['A list of column names']

#--------------------------------------------------------------------------------------------------

#train data
dataset[columns_change_dtype]=dataset[columns_change_dtype].astype(float)


#---------------------------------------------------------------------------------------------------
#score data
df_score[columns_change_dtype]=df_score[columns_change_dtype].astype(float)

<a id='data visualization' style="color:blue;font-size:140%;"> 2. Data Visualization</a>

In [ ]:
# Examining the distribution of the target column which is the historical record of whether the customer made purchase after contact
dataset['TARGET'].value_counts()

In [ ]:
print('Distribution of the Target in thedataset %')
print((dataset['TARGET'].value_counts()/len(dataset)*100))
print(dataset['TARGET'].value_counts())
sns.countplot(x= 'TARGET', data= dataset)
plt.title('TARGET', fontsize=14)
plt.show()

In [ ]:
cat_col=[]    #categorical features

num_col=[]    # numeric features

obj_col=[]    # features with object data type and cardinal more than 10

for col3 in dataset.columns:
    if len(dataset[col3].unique())<=25:
        cat_col.append(col3)
        
    elif len(dataset[col3].unique())>25 and dataset[col3].dtype=="object":
        obj_col.append(col3)
        
    else:
        num_col.append(col3)
        

print("The number of categorical features:", len(cat_col))
print("---------------------------------------------------------------------------------------")
print("The number of numeric features:", len(num_col))
print("---------------------------------------------------------------------------------------")
print("The number of object features(cardinal more than 10):", len(obj_col))

##### 2.2. Visualization of continuous features

<a class="btn btn-warning btn-lg btn-block active" role="button" aria-pressed="true" style="background-color:DodgerBlue; color:white">Violinplot of Continuous Features</a>

In [ ]:
# violinplots for each continuous feature

i=1
plt.figure(figsize=(25,25))
for col in num_col:
    plt.subplot(4,6, i) #!!!
    
    df=pd.melt(dataset[[col]+["TARGET"]], id_vars="TARGET",
    var_name="{}".format(""), value_name='{}_Histogram'.format(col))
    
    sns.violinplot(x="{}".format(""),y="{}_Histogram".format(col), data=df, hue="TARGET",split=True, inner="quart")
    i=i+1

<a class="btn btn-warning btn-lg btn-block active" role="button" aria-pressed="true" style="background-color:DodgerBlue; color:white">Boxplot of Continuous Features</a>

In [ ]:
i=1
plt.figure(figsize=(25,25))
for col in num_col:
    plt.subplot(4,6, i)
    sns.boxplot(x="TARGET", y=col, data=dataset[num_col+["TARGET"]])
    i=i+1

<a class="btn btn-warning btn-lg btn-block active" role="button" aria-pressed="true" style="background-color:DodgerBlue; color:white">Countplot of Categorical Features</a>

In [ ]:
i=1
plt.figure(figsize=(25,75))
for col in cat_col[:60]:
    plt.subplot(10,6, i)
    df_c=dataset[[col,"TARGET"]]
    
    df_c.columns=[col[:50],'TARGET']
    
    a=df_c.columns
    
    df_c[a[0]]=df_c[a[0]].apply(lambda x:str(x)[:50])
    ax=sns.countplot(df_c[a[0]], hue=df_c["TARGET"])
    
    ax.set_xticklabels(
    ax.get_xticklabels(), 
    rotation=45, 
    horizontalalignment='right',
    fontweight='light'
    )
    
    del df_c
    
    i=i+1

<a class="btn btn-warning btn-lg btn-block active" role="button" aria-pressed="true" style="background-color:DodgerBlue; color:white">Correlation Matrix</a>

In [ ]:
corr_matrix = dataset.corr()
fig, ax = plt.subplots(figsize=(25, 15))
ax = sns.heatmap(corr_matrix, annot=True, linewidths=0.5, fmt=".2f", cmap=plt.cm.Accent);
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

In [ ]:
#  Drop highly correlated features(more than 0.95)

cor_matrix_abs = dataset.corr().abs()


upper_tri_abs = cor_matrix_abs.where(np.triu(np.ones(cor_matrix_abs.shape),k=1).astype(np.bool))


features_to_drop = [column for column in upper_tri_abs.columns if any(upper_tri_abs[column] > 0.95)]

<a class="btn btn-warning btn-lg btn-block active" role="button" aria-pressed="true" style="background-color:DodgerBlue; color:white">Negative and positive correlations</a>

In [ ]:
dataset1= dataset.drop(features_to_drop, axis=1)

dataset1.drop('TARGET', axis=1).corrwith(dataset1.TARGET).plot(
    kind='barh', grid=True, figsize=(10, 7),title="Positive & Negative Correlation with the Target", color="r")
plt.xlabel("Correlation with target")
plt.show()

<a id='feature encoding' style="color:blue;font-size:140%;"> 3. Feature encoding</a>

In [ ]:
# A small sample for brute force search
sap_df = dataset1.sample(17000, random_state = 5281)
fig, axes = plt.subplots(nrows=5,ncols=2)


In [ ]:

X_train_fin,X_test_fin, y_train_fin,y_test_fin=train_test_split(data_fin.drop('TARGET',axis=1),data_fin['TARGET'], test_size=0.2,random_state=42)

In [ ]:
def f_encode(train, test,score,col):
    df=pd.concat([train[col],test[col]], axis=0, ignore_index=True).fillna("NaN")
    
    d=df.value_counts(normalize=True).to_dict()
    
    if "missing" in d.keys():
        d["missing"]=-999
        
        train[col]=train[col].fillna("NaN").map(d)
        test[col]=test[col].fillna("NaN").map(d)
        score[col]=score[col].fillna("NaN").map(d)
    else:
        train[col]=train[col].fillna("NaN").map(d)
        test[col]=test[col].fillna("NaN").map(d)
        score[col]=score[col].fillna("NaN").map(d)
    
    return train[col], test[col],score[col]

In [ ]:
for col in dataset_cat_names:
    X_train[col], X_test[col], df_score[col]=f_encode( X_train, X_test,df_score,col)

<a id='Modelling' style="color:blue;font-size:140%;"> 4. Modelling</a>

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE


rus=RandomUnderSampler(sampling_strategy=0.3)

In [ ]:
# This is the main body of a lightgbm model used, parameters are either optimized by grid search or optuna and 
# a list of importance was produced along with all evaulation scores

# Lightgbm with OPTUNA tuned hyperparameters
cv_predictions_lgbm = np.zeros((len(y_train),2))
test_predictions_lgbm = np.zeros((len(y_test),2))
#..................................................................................................................
feature_importance_lgbm=pd.DataFrame()
feature_importance_lgbm["features"]=X_train.drop(rem_col, axis=1).columns.to_list()
feature_importance_lgbm["importance"]=0
#..................................................................................................................
models=dict()
#..................................................................................................................
i=1
skf = StratifiedKFold(n_splits= 11, random_state=42, shuffle=True)
for train_index, test_index in skf.split(X_train.drop(rem_col, axis=1), y_train):
    param_grid = {
        "learning_rate":,
        "n_estimators":,
        "num_leaves":,
        "max_depth":,
        "missing":,
        "is_unbalance":,
        "importance_type":,
        "min_child_samples":,
        "min_split_gain" :,
        "subsample":,
        "colsample_bytree":,
        "reg_alpha":,
        "bagging_fraction":,
        "bagging_freq":
        }
    
    Grid_lgb= HalvingGridSearchCV(LGBMClassifier(n_jobs=-1),scoring ="roc_auc",param_grid=param_grid,
        cv =StratifiedKFold(n_splits=5),refit=True,n_jobs = -1)
    
    X_res, y_res = rus.fit_resample(X_train.drop(rem_col, axis=1).iloc[train_index],y_train.iloc[train_index])
    
    Grid_lgb.fit(X_res, y_res,
                 eval_set=[(X_train.drop(rem_col, axis=1).iloc[train_index],y_train.iloc[train_index]),(X_train.drop(rem_col, axis=1).iloc[test_index],y_train.iloc[test_index])],
            eval_metric=["auc", "binary_logloss"],verbose=True,early_stopping_rounds=200)
                 
    
    clf=Grid_lgb.best_estimator_
    print("---------------------------------------------------------------------------------------------------")
    print("Maodel LGBM "+str(i)+" :",clf)
    models["Maodel LGBM "+str(i)]=clf
    print("---------------------------------------------------------------------------------------------------")
    
    cv_predictions_lgbm[test_index] = clf.predict_proba(X_train.drop(rem_col, axis=1).iloc[test_index])
    test_predictions_lgbm+= clf.predict_proba(X_test.drop(rem_col, axis=1))/ skf.n_splits
    
    feature_importance_lgbm["importance"]+=clf.feature_importances_ /skf.n_splits
    i+=1
#..............................................................................................................   
lgbm_cv_auc_train= roc_auc_score(y_train,cv_predictions_lgbm[:,1])
lgbm_auc_test=roc_auc_score(y_test, test_predictions_lgbm[:,1])
#..............................................................................................................
print("LGBM's Roc CV Score:> ",round(lgbm_cv_auc_train,3))
print("LGBM's Roc Test Score:> ",round(lgbm_auc_test,3))
print("-------------------------------------------------------------------------------------------------------")

<a class="btn btn-warning btn-lg btn-block active" role="button" aria-pressed="true" style="background-color:DodgerBlue; color:white">Classification Report</a>

In [ ]:
#Testing data classification report
print(classification_report(y_test.values, np.where(test_predictions_lgbm[:,1]>0.5,1,0)))

#Training data classification report XGboost without undersampling 2023/09/13
print(classification_report(y_train.values, np.where(cv_predictions_lgbm[:,1]>0.5,1,0)))

<a class="btn btn-warning btn-lg btn-block active" role="button" aria-pressed="true" style="background-color:DodgerBlue; color:white">Confusion Matrix</a>

In [ ]:
plt.figure(figsize=(20,10)) #2023/10/11
plt.subplot(236)

ax1=plt.subplot(231)
skplt.metrics.plot_confusion_matrix(y_test.values,np.where(test_predictions_lgbm[:,1]>0.65,1,0),ax=ax1,title="Confusion Matrix LGBM: Th=0.65",cmap=plt.cm.Blues, normalize=True)

ax2=plt.subplot(232)
skplt.metrics.plot_confusion_matrix(y_test.values,np.where(test_predictions_lgbm[:,1]>0.5,1,0), ax=ax2,title="Confusion Matrix LGBM: Th=0.5",cmap=plt.cm.Accent, normalize=True)

ax3=plt.subplot(233)
skplt.metrics.plot_confusion_matrix(y_test.values,np.where(test_predictions_lgbm[:,1]>0.40,1,0), ax=ax3,title="Confusion Matrix LGBM: Th=0.40",cmap=plt.cm.Blues)

ax4=plt.subplot(234)
skplt.metrics.plot_confusion_matrix(y_test.values,np.where(test_predictions_lgbm[:,1]>0.35,1,0), ax=ax4,title="Confusion Matrix LGBM: Th=0.35",cmap=plt.cm.Accent)

ax5=plt.subplot(235)
skplt.metrics.plot_confusion_matrix(y_test.values,np.where(test_predictions_lgbm[:,1]>0.30,1,0), ax=ax5,title="Confusion Matrix LGBM: Th=0.30",cmap=plt.cm.Blues)

ax6=plt.subplot(236)
skplt.metrics.plot_confusion_matrix(y_test.values,np.where(test_predictions_lgbm[:,1]>0.25,1,0), ax=ax6,title="Confusion Matrix LGBM: Th=0.25",cmap=plt.cm.Accent)

plt.show()

<a class="btn btn-warning btn-lg btn-block active" role="button" aria-pressed="true" style="background-color:DodgerBlue; color:white"> plotting of Metrics</a>

In [ ]:
plt.figure(figsize=(25,20)) 
plt.subplot(236)

ax1=plt.subplot(231)
skplt.metrics.plot_roc_curve(y_test.values,test_predictions_lgbm, ax=ax1,title='LightGBM’s ROC Curves')

ax2=plt.subplot(232)
skplt.metrics.plot_precision_recall_curve(y_test.values,test_predictions_lgbm, ax=ax2,title='LightGBM’s Precision-Recall Curve')

ax3=plt.subplot(233)

skplt.metrics.plot_calibration_curve(y_test.values,[clf.predict_proba(X_test.drop(rem_col, axis=1)) for clf in models.values()],
                                     clf_names=list(models.keys()), ax=ax3,title='LightGBM’s Calibration plots (Reliability Curves)')

ax4=plt.subplot(234)

skplt.metrics.plot_lift_curve(y_test.values,test_predictions_lgbm, ax=ax4,title='LightGBM’s Lift Curve')

ax5=plt.subplot(235)

skplt.metrics.plot_cumulative_gain(y_test.values,test_predictions_lgbm, ax=ax5,title='LightGBM’s Cumulative Gains Curve')


ax6=plt.subplot(236)

skplt.metrics.plot_ks_statistic(y_test.values,test_predictions_lgbm, ax=ax6,title='LightGBM’s KS Statistic Plot')
plt.show()

### Visualisation with plot_metric for a Specific Threshold

In [ ]:

bc_lgbm = functions.BinaryClassification(y_true=y_test.values, y_pred=test_predictions_lgbm[:,1], #2023/08/02
                                    labels=["No", 'Yes'],threshold=0.5)

plt.figure(figsize=(20,17))
plt.subplot2grid(shape=(3,6), loc=(0,0), colspan=2)
bc_lgbm .plot_roc_curve()

plt.subplot2grid((3,6), (0,2), colspan=2)
bc_lgbm .plot_precision_recall_curve()

plt.subplot2grid((3,6), (1,0), colspan=2)
bc_lgbm .plot_class_distribution()

plt.subplot2grid((3,6), (1,2), colspan=2)
bc_lgbm .plot_confusion_matrix(cmap=plt.cm.Accent)

plt.subplot2grid((3,6), (2,0), colspan=2)
bc_lgbm .plot_confusion_matrix(normalize=True,cmap=plt.cm.Blues)

plt.subplot2grid((3,6), (2,2), colspan=2)
bc_lgbm .plot_threshold()

plt.show()
bc_lgbm.print_report()

<a class="btn btn-warning btn-lg btn-block active" role="button" aria-pressed="true" style="background-color:DodgerBlue; color:white">Probability Plot</a>

In [ ]:
plt.figure(figsize=(10,5))
df_preds_lgbm=pd.DataFrame()
df_preds_lgbm["preds_lgbm"]=test_predictions_lgbm[:,1]
df_preds_lgbm["TARGET"]=y_test.values
sns.distplot(df_preds_lgbm[df_preds_lgbm["TARGET"]==1]["preds_lgbm"],color="r",hist=False)
sns.distplot(df_preds_lgbm[df_preds_lgbm["TARGET"]==0]["preds_lgbm"],color="b",hist=False)
plt.title("Probability distribution: LGBM")
plt.xlim((-0.2,1.2))
plt.xlabel("Probability")
plt.legend(["Yes", "No"])
plt.show()

<a class="btn btn-warning btn-lg btn-block active" role="button" aria-pressed="true" style="background-color:DodgerBlue; color:white">Feature Importance</a>

In [ ]:
plt.figure(figsize=(13,10))
y=feature_importance_lgbm.sort_values(by="importance",ascending=False)["importance"][:40].values
x=[c for c in feature_importance_lgbm.sort_values(by="importance",ascending=False)["features"][:40].values]
sns.barplot(x=y,y=x)
plt.title("lgbm’s Feature Importance")
plt.xlabel("Importance")
plt.ylabel("Features")
plt.show()

<a class="btn btn-warning btn-lg btn-block active" role="button" aria-pressed="true" style="background-color:DodgerBlue; color:white">SHAP</a>

In [ ]:
explainer = shap.TreeExplainer(list(models.values())[6])
shap_values = explainer.shap_values(X_test.drop(rem_col, axis=1))

In [ ]:
for name in x:
    shap.dependence_plot(name, shap_values[1], X_test.drop(rem_col,axis=1), display_features=X_test.drop(rem_col,axis=1))

<a id='Prediction' style="color:blue;font-size:140%;"> 4. Prediction</a>

In [ ]:
preds=np.zeros((len(df_score),2))

models=list(pd.read_pickle("PATH").to_dict('index').values())[0]
for model in list(models.keys()):
    model_name=str(model)+'MARKERS'
    loaded_model = pickle.load(open(model_name, 'rb'))
    print(loaded_model)
    preds+=loaded_model.predict_proba(df_score[X_train.columns].drop(rem_col, axis=1))/len(list(models.keys()))
    print('.....................................')
    print("Prediction was made by {}.".format(model))
    print("---------------------------------------------------------------------------------")
    print("---------------------------------------------------------------------------------")
    print("---------------------------------------------------------------------------------")

In [ ]:
pd.DataFrame(data=preds,columns=["No","Yes"]).head(10)

In [ ]:
plt.figure(figsize=(20,6))
plt.subplot(133)


ax1=plt.subplot(131)
sns.countplot(np.where(preds[:,1]>=0.5,1,0),color="b",ax=ax1)
plt.title("Predicted classes for threshold 0.5")


ax2=plt.subplot(132)
sns.distplot(preds[:,1],hist=False,color="r",ax=ax2)
plt.axvline(x=0.5)
plt.title("distribution of probabilities for taking FIO")


ax3=plt.subplot(133)
sns.barplot(x=pd.Series(np.where(preds[:,1]>=0.5,1,0)).value_counts(normalize=True).index,
              y=pd.Series(np.where(preds[:,1]>=0.5,1,0)).value_counts(normalize=True).values,color="g",ax=ax3)
plt.title("Predicted classes(normalized) for threshold 0.5")


plt.show()

In [ ]:
plt.figure(figsize=(15,6))
sns.displot([ i for i in preds[:,1] if i>0.5])
plt.title("Predicted probabilities of exercising (Probabilities > 0.5) vs count")
plt.xlabel("Predicted probabilities")
plt.show();